In [91]:
import random
import numpy as np
import timeit

import pandas as pd
import requests
import time

from citipy import citipy
from config import weather_api_key

# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
coordinates = list (lat_lngs)
coordinates

[(-68.65715366893923, -173.46484831155482),
 (17.15749025809731, -13.295377639259101),
 (-10.903063907253909, 35.257988444208195),
 (-1.402501563219488, 141.63444567392042),
 (-26.527735101009966, 164.98635938949582),
 (-49.18276467743963, -63.58911965621526),
 (25.37139499322747, -118.51106171684565),
 (-31.151854256322018, -113.66427337360082),
 (58.733159750328156, 78.32994822416282),
 (72.43640598198377, -68.7935570541242),
 (-80.57560967507592, -50.14823031580838),
 (77.94368152482491, 156.600633144364),
 (-14.850773818772879, 155.83062422436734),
 (-47.8588256935715, -147.57034861816348),
 (38.037014100709996, -67.57231581280878),
 (-10.524728818859586, -151.5409184309919),
 (-73.94150013834377, 66.22603912314838),
 (-32.48401897886404, -15.782915794623875),
 (78.43881604583797, 172.06055819572214),
 (-50.241927808940915, 65.35283741139747),
 (50.41127964111166, -91.25898173542669),
 (50.6508769663713, 23.859535475031464),
 (-57.44707016044692, 169.00313535760057),
 (57.277440473

In [92]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

757

In [93]:
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

first = 'san francisco'
data_url = base_url + '&q=' + first
data_url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f69231580e8b9810536edd45e8f0a427&q=san francisco'

In [94]:
first_result = requests.get(data_url)
first_result.json()

{'coord': {'lon': -122.4194, 'lat': 37.7749},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 49.62,
  'feels_like': 49.62,
  'temp_min': 42.62,
  'temp_max': 56.25,
  'pressure': 1024,
  'humidity': 84},
 'visibility': 10000,
 'wind': {'speed': 1.01, 'deg': 269, 'gust': 3},
 'clouds': {'all': 20},
 'dt': 1644217288,
 'sys': {'type': 2,
  'id': 2017837,
  'country': 'US',
  'sunrise': 1644160144,
  'sunset': 1644197911},
 'timezone': -28800,
 'id': 5391959,
 'name': 'San Francisco',
 'cod': 200}

In [95]:
# I liked the example in the modules about separating logic in methods.
def parse_result(weather_result):
    city_name = weather_result["name"]
    city_country = weather_result["sys"]["country"]
    city_lat = weather_result["coord"]["lat"]
    city_lng = weather_result["coord"]["lon"]
    city_max_temp = weather_result["main"]["temp_max"]
    city_humidity = weather_result["main"]["humidity"]
    city_clouds = weather_result["clouds"]["all"]
    city_wind = weather_result["wind"]["speed"]
    city_description = weather_result["weather"][0]["description"]

    this_weather_data = {
        "City": city_name,
        "Country": city_country,
        "Lat": city_lat,
        "Lng": city_lng,
        "Max Temp": city_max_temp,
        "Humidity": city_humidity,
        "Cloudiness": city_clouds,
        "Wind Speed": city_wind,
        "Description": city_description,
    }

    return this_weather_data

In [96]:
# Test individual assignments
dummy = first_result.json()
dummy_name = dummy["name"]
dummy_lat = dummy["coord"]["lat"]
dummy_lng = dummy["coord"]["lon"]
dummy_max_temp = dummy["main"]["temp_max"]
dummy_humidity = dummy["main"]["humidity"]
dummy_clouds = dummy["clouds"]["all"]
dummy_wind = dummy["wind"]["speed"]
dummy_description = dummy["weather"][0]["description"]
print(dummy_name, dummy_lat, dummy_lng, dummy_max_temp, dummy_humidity, dummy_clouds, dummy_wind, dummy_description)

San Francisco 37.7749 -122.4194 56.25 84 20 1.01 few clouds


In [97]:
# test parse function
dummy2 = parse_result(dummy)
print(dummy2)

{'City': 'San Francisco', 'Country': 'US', 'Lat': 37.7749, 'Lng': -122.4194, 'Max Temp': 56.25, 'Humidity': 84, 'Cloudiness': 20, 'Wind Speed': 1.01, 'Description': 'few clouds'}


In [98]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Test with subset of cities
subset = cities[0:10]

# use .pop(10) to test loop with first 10 locations
for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = base_url + "&q=" + city.replace(" ","+")

    # Test url generation
    # print(city_url)

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    try:
        data = requests.get(city_url).json()
        api_result = parse_result(data)
        city_data.append(api_result)
    except Exception as e:
        print("Failed with exception: " + str(e))
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 1 | bababe
Failed with exception: 'name'
City not found. Skipping...
Processing Record 3 of Set 1 | kigonsera
Processing Record 4 of Set 1 | vanimo
Processing Record 5 of Set 1 | noumea
Processing Record 6 of Set 1 | comodoro rivadavia
Processing Record 7 of Set 1 | guerrero negro
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | kedrovyy
Processing Record 10 of Set 1 | clyde river
Processing Record 11 of Set 1 | ushuaia
Processing Record 12 of Set 1 | cherskiy
Processing Record 13 of Set 1 | samarai
Processing Record 14 of Set 1 | mataura
Processing Record 15 of Set 1 | nantucket
Processing Record 16 of Set 1 | faanui
Processing Record 17 of Set 1 | taolanaro
Failed with exception: 'name'
City not found. Skipping...
Processing Record 18 of Set 1 | jamestown
Processing Record 19 of Set 1 | pevek
Processing Record 20 of Set 1 | saint-philippe
Pr

In [99]:
city_data

[{'City': 'Vaini',
  'Country': 'TO',
  'Lat': -21.2,
  'Lng': -175.2,
  'Max Temp': 73.56,
  'Humidity': 100,
  'Cloudiness': 100,
  'Wind Speed': 10.36,
  'Description': 'light rain'},
 {'City': 'Kigonsera',
  'Country': 'TZ',
  'Lat': -10.8,
  'Lng': 35.05,
  'Max Temp': 65.5,
  'Humidity': 82,
  'Cloudiness': 100,
  'Wind Speed': 2.08,
  'Description': 'overcast clouds'},
 {'City': 'Vanimo',
  'Country': 'PG',
  'Lat': -2.6741,
  'Lng': 141.3028,
  'Max Temp': 83.01,
  'Humidity': 73,
  'Cloudiness': 100,
  'Wind Speed': 4.14,
  'Description': 'overcast clouds'},
 {'City': 'Noumea',
  'Country': 'NC',
  'Lat': -22.2763,
  'Lng': 166.4572,
  'Max Temp': 78.78,
  'Humidity': 87,
  'Cloudiness': 24,
  'Wind Speed': 14.36,
  'Description': 'few clouds'},
 {'City': 'Comodoro Rivadavia',
  'Country': 'AR',
  'Lat': -45.8667,
  'Lng': -67.5,
  'Max Temp': 76.86,
  'Humidity': 22,
  'Cloudiness': 0,
  'Wind Speed': 26.46,
  'Description': 'clear sky'},
 {'City': 'Guerrero Negro',
  'Countr

In [101]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Vaini,TO,-21.2000,-175.2000,73.56,100,100,10.36,light rain
1,Kigonsera,TZ,-10.8000,35.0500,65.50,82,100,2.08,overcast clouds
2,Vanimo,PG,-2.6741,141.3028,83.01,73,100,4.14,overcast clouds
3,Noumea,NC,-22.2763,166.4572,78.78,87,24,14.36,few clouds
4,Comodoro Rivadavia,AR,-45.8667,-67.5000,76.86,22,0,26.46,clear sky
5,Guerrero Negro,MX,27.9769,-114.0611,58.84,64,0,2.35,clear sky
6,Rikitea,PF,-23.1203,-134.9692,79.12,73,4,16.69,clear sky
7,Kedrovyy,RU,56.1667,91.8167,28.81,78,1,8.84,clear sky
8,Clyde River,CA,70.4692,-68.5914,-37.48,90,72,3.98,broken clouds
9,Ushuaia,AR,-54.8000,-68.3000,54.43,87,100,9.08,overcast clouds


In [103]:
output_filename = "WeatherPy_Database.csv"
city_data_df.to_csv(output_filename, index_label="city_id")